In [1]:
import cv2              #cv2 its default is BGR
import numpy as np
import matplotlib.pyplot as plt    # matplotlib.pyplot its default is RGB
import time
def display_img(img):
    new_img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #cv2.COLOR_BGR2RGB this is called colour code
    plt.imshow(new_img)  #this is done because the plt shows photos by RGB standard so in order to show the photo correctly we changed it from BGR to RGB
    plt.show()

In [3]:
capture = cv2.VideoCapture('song.mp4')
capture_duration = 60 #60seconds
start_time = time.time()

while( int(time.time() - start_time) < capture_duration ):
    is_it_true,frame = capture.read()

    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # extrract blue
    hsv_lower = np.array([110, 50, 50])
    hsv_upper = np.array([130, 255, 255])
    extracted_image = cv2.inRange(hsv_image, hsv_lower, hsv_upper)

    masked_image= cv2.bitwise_and(frame,frame, mask=extracted_image)

    
    cv2.imshow('the song',masked_image)
    
    key=cv2.waitKey(20) #the waitKey function can return the value of a pressed key in addition to wait number of milliseconds
    if key == 27:
        break

capture.release()
cv2.destroyAllWindows()

In [7]:
def extract(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN: 
        colorsB = image[y,x,0] # since the opencv works using BGR principle
        colorsG = image[y,x,1]
        colorsR = image[y,x,2]
        colors  = image[y,x]
        print("Red: ",colorsR)
        print("Green: ",colorsG)
        print("Blue: ",colorsB)
        print("BRG Format: ",colors)

image = cv2.imread('feature1.jpeg')
cv2.namedWindow('colour extraction')
cv2.setMouseCallback('colour extraction',extract)

while(1):
    cv2.imshow('colour extraction',image)
    key=cv2.waitKey(1)
    if key == 27:
        break
#if esc pressed, finish.
cv2.destroyAllWindows()

Red:  66
Green:  56
Blue:  0
BRG Format:  [ 0 56 66]


In [ ]:
import numpy as np


def generate_gaussian_filter(sigma: int | float , filter_shape: list | tuple | None):
    # 'sigma' is the standard deviation of the gaussian distribution

    m, n = filter_shape
    m_half = m // 2
    n_half = n // 2

    # initializing the filter
    gaussian_filter = np.zeros((m, n), np.float32)

    # generating the filter
    for y in range(-m_half, m_half):
        for x in range(-n_half, n_half):
            normal = 1 / (2.0 * np.pi * sigma**2.0)
            exp_term = np.exp(-(x**2.0 + y**2.0) / (2.0 * sigma**2.0))
            gaussian_filter[y+m_half, x+n_half] = normal * exp_term

In [ ]:

from PIL import Image

def convolution(image: np.ndarray , kernel: list | tuple) -> np.ndarray:
   
    if len(image.shape) == 3:
        m_i, n_i, c_i = image.shape

    # if the image is gray then we won't be having an extra channel so handling it
    elif len(image.shape) == 2:
        image = image[..., np.newaxis]
        m_i, n_i, c_i = image.shape
    else:
        raise Exception('Shape of image not supported')

    m_k, n_k = kernel.shape

    y_strides = m_i - m_k + 1  # possible number of strides in y direction
    x_strides = n_i - n_k + 1  # possible number of strides in x direction

    img = image.copy()
    output_shape = (m_i-m_k+1, n_i-n_k+1, c_i)
    output = np.zeros(output_shape, dtype=np.float32)

    count = 0  # taking count of the convolution operation being happening

    output_tmp = output.reshape(
        (output_shape[0]*output_shape[1], output_shape[2])
    )

    for i in range(y_strides):
        for j in range(x_strides):
            for c in range(c_i): # looping over the all channels
                sub_matrix = img[i:i+m_k, j:j+n_k, c]

                output_tmp[count, c] = np.sum(sub_matrix * kernel)

            count += 1

    output = output_tmp.reshape(output_shape)

    return output




def GuassianBlur(img: np.ndarray, sigma: float | int, filter_shape: list | tuple | None = None):
  
    if filter_shape == None:
        # generating filter shape with the sigma(standard deviation) given
        _ = 2 * int(4 * sigma + 0.5) + 1
        filter_shape = [_, _]

    elif len(filter_shape) != 2:
        raise Exception('shape of argument `filter_shape` is not a supported')

    m, n = filter_shape

    m_half = m // 2
    n_half = n // 2

    gaussian_filter = np.zeros((m, n), np.float32)

    for y in range(-m_half, m_half):
        for x in range(-n_half, n_half):
            normal = 1 / (2.0 * np.pi * sigma**2.0)
            exp_term = np.exp(-(x**2.0 + y**2.0) / (2.0 * sigma**2.0))
            gaussian_filter[y+m_half, x+n_half] = normal * exp_term

    blurred = convolution(img, gaussian_filter)

    return gaussian_filter, blurred.astype(np.uint8)


if __name__ == '__main__':
    import matplotlib.pyplot as plt
    from PIL import Image

    img = np.array(Image.open('feature1.jpeg'))
    g_filter, blur_image = GuassianBlur(img, 5, (40, 40))

    # plotting
    plt.subplot(121)
    plt.imshow(g_filter, cmap='gist_heat')  # viridis is also cool
    plt.subplot(122)
    plt.imshow(blur_image)
    plt.tight_layout()
    plt.show()




